# Тюнинг ML моделей

# 0. Импорты

In [ ]:
!pip install lightautoml

In [ ]:
import pandas as pd
import numpy as np

from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestRegressor

# 1. загрузка датасета (регрессия)

In [ ]:
housing_dataset = fetch_california_housing()

In [ ]:
X = pd.DataFrame(housing_dataset.data, columns=housing_dataset.feature_names)
y = pd.Series(housing_dataset.target, name=housing_dataset.target_names[0])

In [ ]:
print(X.shape)
X.head()

(20640, 8)


MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  
0    -122.23  
1    -122.22  
2    -122.24  
3    -122.25  
4    -122.25

In [ ]:
y.head()

0    4.526
1    3.585
2    3.521
3    3.413
4    3.422
Name: MedHouseVal, dtype: float64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape

((16512, 8), (4128, 8))

# 2. Дефолтная модель

In [ ]:
default_model = RandomForestRegressor().fit(X_train, y_train)

In [ ]:
default_model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
r2_default_model = r2_score(y_test, default_model.predict(X_test))
mean_sq_err_defaul_model = mean_squared_error(y_test, default_model.predict(X_test))

print('R2=', np.round(r2_default_model, 3))
print('MSE=', np.round(mean_sq_err_defaul_model, 3))

R2= 0.807
MSE= 0.253


# 3. Ручной подбор параметров

In [ ]:
%%time
manual_tuned_model = RandomForestRegressor(
    n_estimators=150,
    n_jobs=-1,
    max_depth=300,
    min_samples_split=10,
).fit(X_train, y_train)

CPU times: user 15.7 s, sys: 74.8 ms, total: 15.8 s
Wall time: 10.1 s


In [ ]:
r2_manual_model = r2_score(y_test, default_model.predict(X_test))
mean_sq_err_manual_model = mean_squared_error(y_test, default_model.predict(X_test))

print('R2=', np.round(r2_manual_model, 3))
print('MSE=', np.round(mean_sq_err_manual_model, 3))

R2= 0.807
MSE= 0.253


# 4. Grid Search CV

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
%%time
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, None, 150, 200, 300],
}

grid_search = GridSearchCV(
    estimator=RandomForestRegressor(),
    param_grid=param_grid,
    cv=3,
    scoring='r2',
)
grid_search.fit(X_train, y_train)

CPU times: user 6min 45s, sys: 3.01 s, total: 6min 48s
Wall time: 7min 1s


GridSearchCV(cv=3, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [10, None, 150, 200, 300],
                         'n_estimators': [50, 100, 150]},
             scoring='r2')

In [ ]:
grid_search.best_params_

{'max_depth': None, 'n_estimators': 150}

In [ ]:
r2_grid_search = r2_score(y_test, grid_search.predict(X_test))
mean_sq_err_grid_search = mean_squared_error(y_test, grid_search.predict(X_test))

print('R2=', np.round(r2_grid_search, 3))
print('MSE=', np.round(mean_sq_err_grid_search, 3))

R2= 0.807
MSE= 0.253


# 5. Bayesian optimization

In [ ]:
!pip install bayesian-optimization

In [ ]:
%%time
from bayes_opt import BayesianOptimization

def rf_cv(n_estimators, max_depth, min_samples_split):
    val = cross_val_score(
        RandomForestRegressor(
              n_estimators=int(n_estimators),
              max_depth=int(max_depth),
              min_samples_split=int(min_samples_split),
            ),
        X_train,
        y_train,
        cv=3,
        scoring='r2',
    ).mean()
    return val

bayesian_optimization = BayesianOptimization(
    f=rf_cv,
    pbounds={
        'n_estimators': (90, 160),
        'max_depth': (10, 30),
        'min_samples_split': (20, 40),
    },
)
bayesian_optimization.maximize(init_points=10, n_iter=200)

|   iter    |  target   | max_depth | min_sa... | n_esti... |
-------------------------------------------------------------
| 1         | 0.7934    | 23.91     | 20.19     | 148.2     |
| 2         | 0.7898    | 14.53     | 25.67     | 126.0     |
| 3         | 0.7785    | 11.53     | 34.65     | 138.8     |
| 4         | 0.7716    | 10.5      | 37.44     | 146.5     |
| 5         | 0.7909    | 27.16     | 24.34     | 117.7     |
| 6         | 0.7885    | 13.06     | 24.13     | 151.1     |
| 7         | 0.7926    | 15.15     | 20.68     | 144.7     |
| 8         | 0.7941    | 27.22     | 20.21     | 118.5     |
| 9         | 0.7876    | 17.33     | 29.87     | 103.9     |
| 10        | 0.7832    | 18.13     | 38.29     | 153.0     |
| 11        | 0.7931    | 30.0      | 20.0      | 132.8     |
| 12        | 0.7901    | 30.0      | 26.61     | 160.0     |
| 13        | 0.7925    | 30.0      | 20.0      | 90.0      |
| 14        | 0.7821    | 30.0      | 40.0      | 90.0      |
| 15    

In [ ]:
print("Лучшие параметры: ", bayesian_optimization.max['params'])
print("Максимальное значение R^2: ", bayesian_optimization.max['target'])

Лучшие параметры:  {'max_depth': 28.224887677587024, 'min_samples_split': 20.0, 'n_estimators': 158.6584062614922}
Максимальное значение R^2:  0.7950016272669703


# 7. LightAutoML*

In [ ]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task

In [ ]:
%%time
x_y_train = X_train.copy()
x_y_train['target_values'] = y_train

automl = TabularAutoML(
    task = Task(
        name = 'reg',
        metric = lambda y_true, y_pred: r2_score(y_true, y_pred))
)
oof_pred = automl.fit_predict(
    x_y_train,
    roles = {'target': 'target_values'},
)

INFO:lightautoml.automl.presets.base:Stdout logging level is ERROR.
INFO:lightautoml.automl.presets.base:Task: reg

INFO:lightautoml.automl.presets.base:Start automl preset with listed constraints:
INFO:lightautoml.automl.presets.base:- time: 3600.00 seconds
INFO:lightautoml.automl.presets.base:- CPU: 4 cores
INFO:lightautoml.automl.presets.base:- memory: 16 GB

INFO:lightautoml.reader.base:Train data shape: (16512, 9)

INFO3:lightautoml.reader.base:Feats was rejected during automatic roles guess: []
INFO:lightautoml.automl.base:Layer 1 train process start. Time left 3583.73 secs
INFO:lightautoml.ml_algo.base:Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
DEBUG:lightautoml.ml_algo.base:Training params: {'tol': 1e-06, 'max_iter': 100, 'cs': [1e-05, 5e-05, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000], 'early_stopping': 2, 'categorical_idx': [], 'embed_sizes': (), 'data_size': 11}
INFO2:lightautoml.ml_algo.base:===== Start wo

CPU times: user 19min 58s, sys: 50.4 s, total: 20min 49s
Wall time: 13min 14s


In [ ]:
automl_predictions = automl.predict(X_test)
automl_predictions

array([[0.50617456],
       [0.9156231 ],
       [5.246754  ],
       ...,
       [5.003676  ],
       [0.6707553 ],
       [1.7568661 ]], dtype=float32)

In [ ]:
automl_predictions = pd.Series(automl_predictions.data[:, 0])

In [ ]:
r2_automl = r2_score(y_test, automl_predictions)
mean_sq_err_automl = mean_squared_error(y_test, automl_predictions)

print('R2=', np.round(r2_automl, 3))
print('MSE=', np.round(mean_sq_err_automl, 3))

R2= 0.858
MSE= 0.186
